# 데이터 불러오기

In [1]:
# 로컬일 경우 아래 코드 실행 x
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import json

def load_labels(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

train_labels = load_labels('/content/drive/MyDrive/project/KDT_kiosk/face_crop/label/labels.json')

In [ ]:
train_labels

In [25]:
# 수정할 경로
old_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/img/'
new_path = '/content/drive/MyDrive/project/KDT_kiosk/face_crop/img/'

# 경로 수정
for item in train_labels:
    if 'image' in item:
        item['image'] = item['image'].replace(old_path, new_path)

In [ ]:
train_labels

In [ ]:
import os
import shutil

def organize_images_from_json(json_data, img_base_path, output_base_path):
    # 디렉토리 생성
    os.makedirs(output_base_path, exist_ok=True)

    # 모든 인종 클래스 추출
    race_classes = set(item['race'] for item in json_data)
    for cls in race_classes:
        os.makedirs(os.path.join(output_base_path, cls), exist_ok=True)

    # 이미지 이동 또는 복사
    for item in json_data:
        img_path = item['image']
        race = item['race']

        # 이미지 파일 경로 수정
        img_name = os.path.basename(img_path)
        new_img_path = os.path.join(output_base_path, race, img_name)

        # 이미지 파일이 존재하는지 확인
        if os.path.isfile(img_path):
            shutil.copy(img_path, new_img_path)
        else:
            print(f"Warning: {img_path} not found.")

# 경로 설정
img_base_path = '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/img'
output_base_path = '/content/drive/MyDrive/project/KDT_kiosk/organized_dataset'

# 이미지 정리 함수 호출
organize_images_from_json(train_labels, img_base_path, output_base_path)

# dataset을 불러온 다음 과정

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# 사전 훈련된 VGG16 모델 로드
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 커스텀 레이어 추가
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
race_output = Dense(len(race_classes), activation='softmax')(x)  # 클래스 수

model = Model(inputs=base_model.input, outputs=race_output)

# 모델 컴파일
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# 모델 저장
model.save('racial_classification_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# 커스텀 인종 분류 모델 로드
racial_model = load_model('racial_classification_model.h5')

In [ ]:
import cv2
import numpy as np
from deepface import DeepFace

def analyze_with_custom_race(image_path):
    # 이미지 로드
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # DeepFace를 사용해 얼굴 특징 추출
    analysis = DeepFace.analyze(image_path, actions=['emotion', 'age', 'gender'])

    # 커스텀 인종 분류 모델에 맞게 이미지 전처리
    img_resized = cv2.resize(img, (224, 224)) / 255.0
    img_array = np.expand_dims(img_resized, axis=0)

    # 인종 예측
    predictions = racial_model.predict(img_array)
    race_class = np.argmax(predictions[0])

    # 분석 결과에 인종 정보 추가
    race_classes = ['oriental', 'caucasian', 'black', 'hispanic']  # 실제 클래스에 맞게 업데이트
    analysis['race'] = race_classes[race_class]

    return analysis

# 예시 사용
result = analyze_with_custom_race('/path/to/your/image.jpg')
print(result)

# 기존 deepface만으로 했을때 비교 예정